# Segmenting and Clustering Neighbourhoods in Torongo

A Coursera Data Science Capstone Assignment

In [2]:
#import necessary modules
import pandas as pd
import numpy as np

### 1. Web Scraping for Toronto Neighbourhood Data Set

The data will be scraped from wikipedia at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
#install needed packages
!pip install lxml html5lib beautifulsoup4

In [56]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(URL)

In [25]:
print('There are {} tables on the page'.format(len(dfs))) #this show that there are 

There are 3 tables on the page


In [57]:
df = dfs[0] #Inspection shows that our table of interest is the first
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Renaming column Postal Code to PostalCode

In [58]:
df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [59]:
df.drop(df[df.Borough=='Not assigned'].index,inplace=True)
df.index = range(len(df))
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.  These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [60]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [64]:
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: df['Borough'] if x == 'Not Assigned' else x)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Assumptions: 
- The website is available
- The table of interest is available as the first on the page - index 0
- The Schema of the tables are assumed consistent as-is

In [65]:
df.shape

(103, 3)